# Brasil documents analysis

### With Mongodb
Only disc

In [69]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth',200)

In [64]:
br=pd.read_excel('/home/restrepo/colav/7.Datos/LA/Brasil/Original/Publicacoes-periodicos_2013-2020-Scopus.xlsx',nrows=10)

In [66]:
br.columns

Index(['Authors', 'Author(s) ID', 'Title', 'Year', 'Source title', 'Volume',
       'Issue', 'Art. No.', 'Page start', 'Page end', 'Page count', 'Cited by',
       'DOI', 'Link', 'Affiliations', 'Authors with affiliations', 'Abstract',
       'Author Keywords', 'Index Keywords', 'Molecular Sequence Numbers',
       'Chemicals/CAS', 'Tradenames', 'Manufacturers', 'Funding Details',
       'Funding Text 1', 'Funding Text 2', 'Funding Text 3', 'Funding Text 4',
       'Funding Text 5', 'References', 'Correspondence Address', 'Editors',
       'Sponsors', 'Publisher', 'Conference name', 'Conference date',
       'Conference location', 'Conference code', 'ISSN', 'ISBN', 'CODEN',
       'PubMed ID', 'Language of Original Document',
       'Abbreviated Source Title', 'Document Type', 'Publication Stage',
       'Access Type', 'Source', 'EID', 'ID bolsistas'],
      dtype='object')

In [84]:
do=br[['Authors',  'Title', 'Year', 'Source title', 'Volume',
       'Issue',  'Page start', 'Page end','DOI', 
       'Author Keywords', 'ISSN',  'ID bolsistas']].loc[0].to_dict()
do

{'Authors': 'SANCHES R., RAZKY A.',
 'Title': 'Variação Linguística para Cigarro de Palha e Toco de Cigarro no Atlas Linguístico do Amapá',
 'Year': 2015,
 'Source title': 'Revista Todas as Letras (MACKENZIE. Online)',
 'Volume': 17.0,
 'Issue': nan,
 'Page start': 196,
 'Page end': 206.0,
 'DOI': '10.15529/1980-6914/letras.v17n3p196-206',
 'Author Keywords': nan,
 'ISSN': 19806914,
 'ID bolsistas': "'8153913927369006"}

In [24]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import sys

In [2]:
client = MongoClient()

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
#client.drop_database('la')

In [5]:
client.list_database_names()

['br', 'la', 'local']

In [6]:
db = client['br']

In [7]:
db.list_collection_names()

['stage_cites',
 'stage_cites_invalid',
 'quarantine',
 'stage_invalid',
 'gslookup_not_found',
 'data',
 'cache_cites',
 'stage']

In [8]:
dt=db.get_collection('data')
st=db.get_collection('stage')

In [39]:
st.count_documents({})

418177

In [187]:
400%100

0

In [190]:
j=0
br_new=pd.DataFrame()
#cursor timeout: https://stackoverflow.com/a/24200795/2268280
cursor=dt.find(no_cursor_timeout=True)
for d in cursor:
    i=d.get("article_id")
    #print(f'id: {i}')
    l=[d for d in st.find({'article_id':str(i)})]
    if l:
        n=l[0].get('cites')
        if not n:
            n=0
        d['cites']=n
    else:
        d['cites']=0
    kk=[ d.pop(k) for k in 
        ['_id','publisher', 'country', 'abstract', 'language', 'article_id'] 
       ]
    br_new=br_new.append(d,ignore_index=True)
    if j%100==0:
        print(str(j).zfill(6),end='\r')
    j+=1
    #if j==20:
    #    break
cursor.close()    

In [206]:
def check_page(p):
    if isinstance(p,str):
        return p
    else:
        return str(int(p))
    
br_new['ISSN']=br_new['ISSN'].apply(lambda x: str(int(x)).zfill(8) if isinstance(x,float) else str(x).zfill(8) )
br_new['Page end']=br_new['Page end'].apply(lambda x: check_page(x) if not pd.isna(x) else None)
br_new['Page start']=br_new['Page start'].apply(lambda x: check_page(x) if not pd.isna(x) else None)
br_new['volume']=br_new['volume'].apply(lambda x: check_page(x) if not pd.isna(x) else None)
br_new=br_new.astype({'year': int,'cites':int})

In [213]:
br_new.to_excel('Publicacoes-periodicos_2013-2020-Scopus-con-citas.xlsx',index=False)

In [215]:
!ls -lh Publicacoes-periodicos_2013-2020-Scopus-con-citas.xlsx

-rw-r--r-- 1 restrepo restrepo 78M Oct 19 22:31 Publicacoes-periodicos_2013-2020-Scopus-con-citas.xlsx


In [4]:
import wosplus as wp
import pandas as pd

In [2]:
%%writefile drive.cfg
[FILES]
Publicacoes-periodicos_2013-2020-Scopus-con-citas_brasil.xlsx=104XwW61YaPA30KLsFBogTbX4wfg1LQGP

Writing drive.cfg


In [3]:
gd=wp.wosplus('drive.cfg')

In [8]:
br=gd.read_drive_excel('Publicacoes-periodicos_2013-2020-Scopus-con-citas_brasil.xlsx')

In [21]:
df=pd.DataFrame( br.apply(lambda row: [{'ID':row.get('ID'),'author':x} for x in row.get('author').split(', ')],axis=1
             ).apply(pd.Series).stack().to_list())

In [26]:
df.to_csv('authors.csv',index=False)

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('authors.csv')

In [6]:
df=df.reset_index(drop=True)

In [7]:
df.shape

(8034544, 2)

In [12]:
import numpy as np

In [19]:
r=range(0,10000000,1000000)

In [26]:
df[2000000:3000000]

,ID,author
2000000,57678,Hos I.
2000001,57678,Kangal E.E.
2000002,57678,Kayis Topaksu A.
2000003,57678,Onengut G.
2000004,57678,Ozdemir K.
...,...,...
2999995,79510,Iodice M.
2999996,79510,Iordanidou K.
2999997,79510,Ippolito V.
2999998,79510,Irles Quiles A.


In [27]:
for i in range(len(r)):
    print(i,'→',r[i],r[i+1])
    df[r[i]:r[i+1]].to_excel(f'authors_{i+1}.xlsx',index=False)
    if i==len(r)-2:
        break

0 → 0 1000000
1 → 1000000 2000000
2 → 2000000 3000000
3 → 3000000 4000000
4 → 4000000 5000000
5 → 5000000 6000000
6 → 6000000 7000000
7 → 7000000 8000000
8 → 8000000 9000000


In [27]:
ls -lh authors.csv

-rw-r--r-- 1 restrepo restrepo 144M Oct 20 19:31 authors.csv


In [1]:
df['ID'].unique().shape

NameError: name 'df' is not defined

In [31]:
df['ID'].value_counts?

Object `value_counts` not found.


In [32]:
df['ID'].value_counts()

295104    2680
34323     2673
240399    2655
57472     2652
57476     2651
          ... 
334166       1
334165       1
334157       1
334153       1
409020       1
Name: ID, Length: 437952, dtype: int64

In [ ]:
df['ID'].values_counts